1. Carregar os dados dos discursos

In [18]:
import pandas as pd
import sqlite3

# Caminho ao banco de dados SQLite
db_path = "../../../DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco
conn = sqlite3.connect(db_path)

# Carregar os dados da tabela 'AnaliseCorpusTodo'
query = """
SELECT
    a.*,
    d.TextoResumo,
    d.Indexacao,
    d.DataPronunciamento,
    d.SiglaCasaPronunciamento,
    s.NomeParlamentar,
    s.CodigoParlamentar,
    d.SiglaPartidoParlamentarNaData,
    d.UfParlamentarNaData

FROM AnaliseCorpusTodo AS a
LEFT JOIN Discursos AS d
    ON a.CodigoPronunciamento = d.CodigoPronunciamento
LEFT JOIN Senadores AS s
    ON d.CodigoParlamentar = s.CodigoParlamentar

WHERE d.SiglaCasaPronunciamento = 'SF'
"""
df_discursos = pd.read_sql_query(query, conn)

# Fechar conexão
conn.close()

print(df_discursos.head())


   CodigoPronunciamento                                      AvalConclusao  \
0                366328                                               None   
1                366329                                               None   
2                366330                                               None   
3                366331  Por causa disso, o orador defende a necessidad...   
4                366332                                               None   

                                      AvalImplicacao  \
0                                               None   
1                                               None   
2                                               None   
3  isso significa que a Constituição busca promov...   
4                                               None   

                                      AvalPredicacao  \
0                                               None   
1                                               None   
2                         

2. Definir coluna de análise

In [19]:
coluna = input("Digite a coluna da tabela AnaliseCorpusTodo a ser topicalizada. Caso seja mais de uma coluna, prepare uma coluna combinada.")

3. Carregar embeddings FAISS e os códigos

In [20]:
import numpy as np
import pandas as pd
import faiss

# Caminhos aos arquivos
index_path = f"../../../../data/discursos/embeddings/discursos_{coluna}.index"
codigos_path = f"../../../../data/discursos/embeddings/codigos_{coluna}.npy"

# Carregar códigos (ids dos documentos)
codigos = np.load(codigos_path)

# Carregar índice FAISS e extrair embeddings
faiss_index = faiss.read_index(index_path)
embeddings = faiss_index.reconstruct_n(0, faiss_index.ntotal)


4. Verificação e alinhamento embeddings ↔ textos originais

In [21]:
# Validação rápida inicial
print("Embeddings shape:", embeddings.shape)
print("Codigos shape:", codigos.shape)

# Criar DataFrame para alinhamento seguro
df_embeddings = pd.DataFrame({
    "CodigoPronunciamento": codigos,
    "embedding_index": np.arange(len(codigos))
})

# Garantir alinhamento com df_final_valido (já preparado anteriormente)
df_final_valido = pd.merge(
    df_discursos, df_embeddings,
    on="CodigoPronunciamento", how="inner"
)

# Selecionar embeddings válidos
embeddings_validos = embeddings[df_final_valido["embedding_index"]]

# Validação final
print(f"Nº documentos válidos alinhados: {len(df_final_valido)}")
print(f"Shape embeddings válidos alinhados: {embeddings_validos.shape}")


Embeddings shape: (15531, 3072)
Codigos shape: (15531,)
Nº documentos válidos alinhados: 15142
Shape embeddings válidos alinhados: (15142, 3072)


In [22]:
df_final_valido.head()

,CodigoPronunciamento,AvalConclusao,AvalImplicacao,AvalPredicacao,AvalTrecho,BM25_CorpusTodo_Constituição,MencionaConstituicao,NormConclusao,NormImplicacao,NormPredicacao,...,AvalCombinado,NormCombinado,TextoResumo,Indexacao,DataPronunciamento,SiglaCasaPronunciamento,NomeParlamentar,CodigoParlamentar,SiglaPartidoParlamentar,embedding_index
0,366329,None,None,None,None,1.414732,0.0,None,None,None,...,. . .,. . .,Desafios do Programa de Aceleração do Crescime...,"ELOGIO, PROGRAMA, ACELERAÇÃO, CRESCIMENTO, EXP...",2007-02-07,SF,Antonio Carlos Valadares,3,PSB,2241
1,366331,"Por causa disso, o orador defende a necessidad...",isso significa que a Constituição busca promov...,a Constituição foi concebida para aumentar a p...,O espírito da Constituição foi exatamente aume...,2.007791,1.0,"Por causa disso, o orador critica a prática do...",isso implica que o Governo Federal deve respei...,a Constituição estabelece o pacto federativo q...,...,a Constituição foi concebida para aumentar a p...,a Constituição estabelece o pacto federativo q...,Críticas ao recém-lançado Programa de Aceleraç...,"COMENTARIO, ANUNCIO, PROGRAMA, ACELERAÇÃO, CRE...",2007-02-07,SF,Cícero Lucena,4529,PSDB,2242
2,366333,"Por causa disso, o orador defende a implementa...",Isso significa que o atual modelo de orçamento...,a Constituição é insuficiente para garantir a ...,Um país da dimensão do Brasil não pode mais te...,1.060610,1.0,"Por causa disso, o orador propõe uma emenda co...",Isso implica que qualquer alteração na arrecad...,a Constituição estabelece a base para o cálcul...,...,a Constituição é insuficiente para garantir a ...,a Constituição estabelece a base para o cálcul...,Ratifica compromissos assumidos com a populaçã...,"REGISTRO, DADOS, REDUÇÃO, FUNDO DE PARTICIPAÇÃ...",2007-02-07,SF,Francisco Dornelles,765,PP,2243
3,366336,None,None,None,None,1.739727,0.0,None,None,None,...,. . .,. . .,Afirmações de que o Programa de Aceleração do ...,"BALANÇO, EFICACIA, ACORDO, SENADO, INICIO, ANO...",2007-02-07,SF,Ideli Salvatti,3373,PT,2244
4,366337,None,None,None,None,0.000000,NaN,None,None,None,...,. . .,. . .,Defesa da condução do ex-Presidente do Supremo...,"REGISTRO, IMPORTANCIA, ATUAÇÃO, PARTIDO POLITI...",2007-02-07,SF,Pedro Simon,68,MDB,2245


5. Garantir que datas estejam no formato adequado

In [23]:
df_final_valido["DataPronunciamento"] = pd.to_datetime(df_final_valido["DataPronunciamento"], errors='coerce')

# Checar problemas de datas
print(df_final_valido["DataPronunciamento"].isnull().sum())


0


In [24]:
len(df_final_valido)

15142

6. Salvar dataset preparado

In [25]:
# Exportar resultados finais de maneira clara e definitiva
df_final_valido.to_parquet(f"dataset_preparado_{coluna}.parquet", index=False)
np.save(f"embeddings_validos_{coluna}.npy", embeddings_validos)